In [6]:
import numpy as np

# Stałe
LAMBDA = 256           # Poziom bezpieczeństwa algorytmu
SEEDSIZE = 48          # Rozmiar nasienia używanego do generowania losowych wartości
LOG2_Q = 50            # Logarytm dwójkowy z Q
N = 80                 # Liczba zmiennych
B = 6                  # Liczba bitów ekstrahowanych z elementu
M = 81                 # Liczba równań
Q = 1125899906842624   # Moduł Q (2^50)
COFSIZE = 4096         # Górna granica rozmiaru współczynników wielomianów
SECRETVAL_LENGTH = 1   # Długość sekretnej wartości
SHAREDKEYSIZE = (M * B // 8)  # Rozmiar współdzielonego klucza
ERROR_LENGTH = 1       # Długość błędu
PK_LENGTH = (M * 8)    # Rozmiar klucza publicznego
RANGE = 7              # Zakres wartości
B_BAR = (LOG2_Q - B)   # Liczba bitów do zaokrąglenia

class Pol:
    """Klasa reprezentująca wielomian z współczynnikami QD, L i C"""
    def __init__(self, n):
        self.QD = np.zeros((n, n), dtype=np.int64)
        self.L = np.zeros(n, dtype=np.int64)
        self.C = 0

# Funkcja generująca losowe bajty
def randombytes(size):
    return np.random.randint(0, 256, size, dtype=np.uint8)

# Funkcja generująca wielomiany z losowymi współczynnikami
def polgen(f, m, n):
    for i in range(m):
        cofval = randombytes((N * (N + 1) // 2) + N + 1) % COFSIZE
        count = 0
        
        for j in range(n):
            for k in range(n):
                if k > j:
                    f[i].QD[j, k] = 0
                else:
                    f[i].QD[j, k] = cofval[count] % COFSIZE
                    count += 1

        for j in range(n):
            f[i].L[j] = cofval[count] % COFSIZE
            count += 1

        f[i].C = cofval[count] % COFSIZE

# Funkcja obliczająca wartość wielomianu dla danego pValue
def evaluate_poly(unPoly, pValue, n):
    result1 = 0
    result2 = 0

    for j in range(n):
        tabResult1 = np.dot(pValue, unPoly.QD[:, j])
        result1 += tabResult1 * pValue[j]

    result2 = np.dot(unPoly.L, pValue)
    result1 += result2 + unPoly.C

    return result1 % Q

# Funkcja oceniająca system wielomianów dla danego pValue
def Eval_sys(pSyst, pValue, m, n):
    return np.array([evaluate_poly(pSyst[i], pValue, n) for i in range(m)])

# Funkcja realizująca pierwszy etap cross-roundingu w algorytmie KEM
def kem_crossround1(in_value):
    return (int(in_value) >> (B_BAR - 1)) % 2

# Funkcja zaokrąglająca wartość wejściową
def rounding(in_value):
    rem = (int(in_value) + (1 << (B_BAR - 1))) % Q
    return rem >> B_BAR

# Funkcja realizująca drugi etap cross-roundingu dla tablicy wartości w algorytmie KEM
def kem_crossround2(in_values):
    return np.array([(int(val) >> (B_BAR - 1)) % 2 for val in in_values], dtype=np.uint8)

# Funkcja zaokrąglająca wartości wejściowe
def kem_rounding(in_values):
    return np.array([(int(val) + (1 << (B_BAR - 1))) % Q >> B_BAR for val in in_values], dtype=np.uint8)

# Funkcja rekonstruująca klucz na podstawie wartości w i c w algorytmie KEM
def kem_rec(key, w, c):
    for i in range(len(w)):
        hint = kem_crossround1(w[i])
        if hint == c[i]:
            key[i] = rounding(w[i])
        else:
            w1 = (int(w[i]) + (1 << (B_BAR - 2)) - 1)
            hint = kem_crossround1(w1)
            if hint == c[i]:
                key[i] = rounding(w1)
            else:
                w2 = (int(w[i]) - (1 << (B_BAR - 2)) + 1)
                hint = kem_crossround1(w2)
                if hint == c[i]:
                    key[i] = rounding(w2)
                else:
                    key[i] = rounding(w[i])

# Funkcja pakująca klucz tajny
def pack_sk(sk, sa, seed):
    sk[:SEEDSIZE] = seed
    sk[SEEDSIZE:] = sa

# Funkcja rozpakowująca klucz tajny
def unpack_sk(sk):
    seed = sk[:SEEDSIZE]
    sa = sk[SEEDSIZE:]
    return sa, seed

# Funkcja pakująca klucz publiczny
def pack_pk(pk, b1, seed):
    pk[:SEEDSIZE] = seed
    for i in range(M):
        pk[SEEDSIZE + i*8:SEEDSIZE + (i+1)*8] = np.flip(np.frombuffer(np.uint64(b1[i]).tobytes(), dtype=np.uint8))

# Funkcja rozpakowująca klucz publiczny
def unpack_pk(pk):
    seed = pk[:SEEDSIZE]
    b1 = np.zeros(M, dtype=np.uint64)
    for i in range(M):
        b1[i] = int.from_bytes(np.flip(pk[SEEDSIZE + i*8:SEEDSIZE + (i+1)*8]), 'big')
    return b1, seed

# Funkcja pakująca tekst zaszyfrowany
def pack_ct(ct, b2, c):
    ct[:M] = c
    for i in range(M):
        ct[M + i*8:M + (i+1)*8] = np.flip(np.frombuffer(np.uint64(b2[i]).tobytes(), dtype=np.uint8))

# Funkcja rozpakowująca tekst zaszyfrowany
def unpack_ct(ct):
    c = ct[:M]
    b2 = np.zeros(M, dtype=np.uint64)
    for i in range(M):
        b2[i] = int.from_bytes(np.flip(ct[M + i*8:M + (i+1)*8]), 'big')
    return b2, c

# Funkcja generująca parę kluczy KEM (publiczny i tajny)
def crypto_kem_keypair():
    seed = randombytes(SEEDSIZE)
    np.random.seed(seed)

    f1 = allocatemem(M, N)
    polgen(f1, M, N)

    sa = randombytes(N) % RANGE
    e1 = randombytes(M) % RANGE

    b1 = Eval_sys(f1, sa, M, N)
    b1 = (b1 + e1) % Q

    sk = np.zeros(SEEDSIZE + N, dtype=np.uint8)
    pack_sk(sk, sa, seed)

    pk = np.zeros(SEEDSIZE + M * 8, dtype=np.uint8)
    pack_pk(pk, b1, seed)

    return pk, sk

# Funkcja enkapsulująca klucz (szyfrowanie)
def crypto_kem_enc(pk):
    b1, seed = unpack_pk(pk)
    np.random.seed(seed)

    f2 = allocatemem(M, N)
    polgen(f2, M, N)

    seed1 = randombytes(SEEDSIZE)
    np.random.seed(seed1)

    sb = randombytes(N) % RANGE
    e2 = randombytes(M) % RANGE
    e3 = randombytes(M) % RANGE

    b2 = Eval_sys(f2, sb, M, N)
    b3 = (b2 * b1 + e3) % Q
    b2 = (b2 + e2) % Q

    ss = kem_rounding(b3)
    c = kem_crossround2(b3)

    ct = np.zeros(M + M * 8, dtype=np.uint8)
    pack_ct(ct, b2, c)

    return ct, ss

# Funkcja dekapsulująca klucz (deszyfrowanie)
def crypto_kem_dec(ct, sk):
    sa, seed = unpack_sk(sk)
    b2, c = unpack_ct(ct)
    np.random.seed(seed)

    f = allocatemem(M, N)
    polgen(f, M, N)

    w = Eval_sys(f, sa, M, N)
    w = (w * b2) % Q

    if np.all(w == 0):
        raise ValueError("Reconstructed w is all zeros, which is incorrect.")

    ss = np.zeros(M, dtype=np.uint8)
    kem_rec(ss, w, c)

    return ss

In [7]:
pk, sk = crypto_kem_keypair()
print("Public Key:", pk)
print("Private Key:", sk)

ct, ss_enc = crypto_kem_enc(pk)
print("Ciphertext:", ct)
print("Shared Secret (Encryption):", ss_enc)

ss_dec = crypto_kem_dec(ct, sk)
print("Shared Secret (Decryption):", ss_dec)

assert np.array_equal(ss_enc, ss_dec), "Shared secrets do not match!"
print("Shared secrets match!")

Public Key: [ 39  30 246 238 130 226   2  94 168 158  76 102 244 240   7 178  94 159
 115   2 119  80 189 101 177  13 190  90  18 203   6   6 195 225  32 188
 148   7 196 225 228 215 129 229 237 254  41 176   0   0   0   0   0  63
   0 221   0   0   0   0   0  63  65 248   0   0   0   0   0  63  68 185
   0   0   0   0   0  62  80   2   0   0   0   0   0  64 109 159   0   0
   0   0   0  63  99 182   0   0   0   0   0  62   0  38   0   0   0   0
   0  63 153  97   0   0   0   0   0  62 193 152   0   0   0   0   0  62
 138  25   0   0   0   0   0  64  75 177   0   0   0   0   0  61 252 201
   0   0   0   0   0  64  28 185   0   0   0   0   0  64  56  47   0   0
   0   0   0  63  46  30   0   0   0   0   0  62 156 137   0   0   0   0
   0  62 163 160   0   0   0   0   0  62 155  80   0   0   0   0   0  65
 127  95   0   0   0   0   0  62  12 128   0   0   0   0   0  62 207 109
   0   0   0   0   0  62 228 239   0   0   0   0   0  63  44 180   0   0
   0   0   0  63   2  49   0   0   0   

AssertionError: Shared secrets do not match!